In [ ]:
# Libraries
import random
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams["axes.spines.right"] = False
mpl.rcParams["axes.spines.top"] = False
plt.style.use("seaborn-v0_8-whitegrid")
mpl.rcParams['figure.dpi'] = 100
mpl.rcParams['savefig.dpi'] = 600

# Constants
random.seed(0)
tf = 60
dt = 0.001
N = round(tf/dt)
t = np.arange(0, tf, dt)

c0 = [0, 0]

def x_curve(t, k1, k2, k3, k4, xi):
    # Solves the system for the given parameters analytically

    Ac = 1
    eta = Ac
    
    B = k1 + k4
    C = k1*k4 + k2*k3
    Q = k4 - k2*xi
    
    x_raya = Q/C * Ac
    eig_1 = -(-B + np.sqrt(B**2 - 4*C))/2
    eig_2 = -(-B - np.sqrt(B**2 - 4*C))/2
    C1 = (eta - eig_2*x_raya)/(eig_2-eig_1)
    C2 = (eta - eig_1*x_raya)/(eig_2-eig_1)
    return C1*np.exp(-eig_1*t) - C2*np.exp(-eig_2*t) + x_raya

# NFBL

In [ ]:
# xi = 0

# Initial curve parameters
k1, k2, k3, k4, Ac, xi = 1, 2, 0.05, 0.05, 1, 0

B = k1 + k4
C = k1*k4 + k2*k3
Q = k4

# k4 has to be fixed as 1, therefor k1 also has to be fixed as 1
total_curves = 50000
num_curves = int(total_curves/2)
k2_vals = np.random.uniform(0, 3, num_curves)
k3_vals = (C - k1*k4)/k2_vals

In [ ]:
# Solve the system for each pair of k2 and k3
t = np.arange(0, tf, dt)
t_shape = np.shape(t)[0]

x = np.empty([num_curves, t_shape])
y = np.empty([num_curves, t_shape])

for i in range(num_curves):
    if i+1 % 10000 == 0:
        print(i)
    
    x[i] = x_curve(t, k1, k2_vals[i], k3_vals[i], k4, xi)

In [ ]:
# Plot X
fig, ax = plt.subplots(2, 3, figsize=(20, 10))

for i in range(6):
    ax[i//3, i%3].plot(t, x[i])
    ax[i//3, i%3].set_xlabel('Time [ms]')
    ax[i//3, i%3].set_ylabel('x')

In [ ]:
data = np.empty([num_curves*2, tf])

t_correction = int(1/dt)
indices = np.arange(tf) * t_correction

for i in range(num_curves):
    data[i] = x[i][indices]

# IFFL

In [ ]:
# K3 = 0

# B = 1.05
# C = 0.15
# Q = 0.05

# Initial curve parameters
K4 = (-B + np.sqrt(B**2 - 4*C)) / (-2)
K1 = B - K4
K3 = 0

B = K1 + K4 # -> K1 = B-K4 -> K1 = 0.8794361719689456
C = K1*K4 # -> K4*(B-K4) = C -> 0 = -K4^2 + B*K4 - C -> K4 = 0.1705638280310544
Q = 0.05

# K1 and K4 have to be fixed to their respective values
K2_vals = np.random.uniform(0, 3, num_curves)
xi_vals = (K4-Q)/K2_vals

print(f"K1: {K1}, K4: {K4}, B: {B}, C: {C}, Q: {Q}.")

In [ ]:
# Solve the system for each pair of K2 and xi
x = np.empty([num_curves, t_shape])
y = np.empty([num_curves, t_shape])

for i in range(num_curves):
    if i+1 % 10000 == 0:
        print(i)
    
    x[i] = x_curve(t, K1, K2_vals[i], K3, K4, xi_vals[i])

In [ ]:
# Plot X
fig, ax = plt.subplots(2, 3, figsize=(20, 10))

for i in range(6):
    ax[i//3, i%3].plot(t, x[i])
    ax[i//3, i%3].set_xlabel('Time [ms]')
    ax[i//3, i%3].set_ylabel('x')

In [ ]:
for i in range(num_curves):
    data[num_curves+i] = x[i][indices]

In [ ]:
# Create the labels array
# 1 NFBL, 0 IFFL
labels = np.empty(total_curves)
labels[:num_curves] = 1
labels[num_curves:] = 0

In [ ]:
# Save the data
np.savetxt("data_analytical.csv", data, delimiter=",")
np.savetxt("labels.csv", labels, delimiter=",")